# 00 import module

In [2]:
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

# 01 load data

In [3]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

In [4]:
data_dir = keras.utils.get_file(origin=dataset_url,
                               fname='flower_photos',
                               untar=True)
data_dir = pathlib.Path(data_dir)

# 02 preprocessing

In [5]:
train_ds = keras.preprocessing.image_dataset_from_directory(
data_dir,
validation_split=0.2,
subset='training',
seed=2023,
image_size=(224, 224),
batch_size=32)

test_ds = keras.preprocessing.image_dataset_from_directory(
data_dir,
validation_split=0.2,
subset='training',
seed=2023,
image_size=(224, 224),
batch_size=32)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 2936 files for training.


# 03 data explore

In [ ]:
class_names = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulibs']
plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
plt.tight_layout()

In [ ]:
class CNN(keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.cnn1 = keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(224,224,3))
        self.pool1 = keras.layers.MaxPooling2D((2,2))
        self.cnn2 = keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')
        self.pool2 = keras.layers.MaxPooling2D((2,2))
        
        self.flat = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(128, activation='relu')
        self.out = keras.layers.Dense(5, activation='softmax')
    
    def call(self,x):
        x = self.cnn1(x)
        x = self.pool1(x)
        x = self.cnn2(x)
        x = self.pool2(x)    
        x = self.flat(x)
        x = self.fc1(x)
        
        return self.out(x)

In [8]:
model = CNN()

In [10]:
learning_rate = 1e-4
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True) # multi-label classification 문제라서
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
model.build(input_shape=(1, 224, 224, 3)) # (batch, height, width, channel) - batch는 전체 데이터를 쪼개는 단위, channel은 RGB (여기서는 greyscale이라서 channel이 한 개!)
model.summary()

Model: "cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  896       
                                                                 
 max_pooling2d (MaxPooling2  multiple                  0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           multiple                  18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  multiple                  0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  25690240

In [13]:
model.fit(train_ds, epochs=10)

Epoch 1/10
92/92 [==============================] - 35s 377ms/step - loss: 12.5171 - accuracy: 0.3181
Epoch 2/10
92/92 [==============================] - 35s 382ms/step - loss: 1.3049 - accuracy: 0.4452
Epoch 3/10
92/92 [==============================] - 35s 379ms/step - loss: 1.1741 - accuracy: 0.5208
Epoch 4/10
92/92 [==============================] - 39s 417ms/step - loss: 1.0640 - accuracy: 0.5685
Epoch 5/10
92/92 [==============================] - 35s 376ms/step - loss: 0.9526 - accuracy: 0.6206
Epoch 6/10
92/92 [==============================] - 35s 378ms/step - loss: 0.8704 - accuracy: 0.6614
Epoch 7/10
92/92 [==============================] - 34s 364ms/step - loss: 0.7728 - accuracy: 0.7057
Epoch 8/10
92/92 [==============================] - 34s 365ms/step - loss: 0.7152 - accuracy: 0.7384
Epoch 9/10
92/92 [==============================] - 34s 364ms/step - loss: 0.6330 - accuracy: 0.7704
Epoch 10/10
92/92 [==============================] - 34s 367ms/step - loss: 0.5710 - accur

In [12]:
model.evaluate(test_ds)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


92/92 [==============================] - 10s 110ms/step - loss: 42.9641 - accuracy: 0.1945


[42.96405792236328, 0.19448228180408478]

In [29]:
hist = model_second.fit(train_ds, epochs=1)

92/92 [==============================] - 35s 378ms/step - loss: 0.0775 - accuracy: 0.9932


In [18]:
test_loss, test_accuracy = model.evaluate(test_ds)

print("Test accuracy: ", test_accuracy)
print("Test loss: ", test_loss)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.